In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# modifiy default model of crewAI
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
langtrace_api_key = os.environ.get("LANGTRACE_API_KEY")


In [2]:
# ROE = Return on Equity -> company overview
# PER = Price Earning Ratio -> company overview
# FFO = 당기순이익 - 비영업이익 -> Net Income - OtherNonOperatingIncome    --->   비영업수익 : income_statement

# 배당률 = dividendpayout / net Income

In [3]:
from pydantic import BaseModel
from typing import List

'''
class BlogPost(BaseModel)
    title: str
    introduction: str
    sections: List[SubSection]
'''

class Indicator(BaseModel):
    indicator : float
    info_of_indicator : str

class News(BaseModel):
    news_title : str
    summarize_of_news : str
    analysis : str


class AnalysisReport(BaseModel):
    company: str
    decision : str
    indicator_report : List[Indicator]
    diviend_payout_ratio_report : str
    news : List[News]
    market_share : str
    stock_price : str


In [ ]:
"""output 강제화 -> **langserve** API 화 -> reactJS로 웹 만들기"""

In [5]:
from crewai_tools import tool
import requests
import yfinance as yf


class Tools:
    @tool("Information of Company Indicator")
    def get_information_of_indicator():
        """
        Give information about traditional indicator with analysing company stock.
        """
        return """
        There are three Traditional Indicator you should check. 
        ROE (Return on Equity):
        ROE measures the efficiency and profitability of a company's equity investment. A higher ROE indicates more effective management and profitability, with a benchmark of 15% being desirable. Companies with high ROE are often well-managed and show potential for long-term growth.

        PER (Price Earning Ratio):
        PER assesses if a stock is undervalued or overvalued by comparing the stock price to its EPS. A PER below 1 suggests the stock may be undervalued, while a PER above 1 indicates possible overvaluation. Investing in stocks with a PER below 1 is generally seen as a wise choice.

        FFO (Funds From Operations):
        FFO reflects earnings from core operations by adjusting net income for non-operational gains and expenses. It's a more accurate measure of a company’s operational performance compared to net income. Companies with consistently growing FFO over the past five years are considered good investment opportunities.
        """
    
    @tool("Get ROE(Return on Equity) rate of company")
    def get_ROE(symbol):
        """
        Useful to get ROE(Return on Equity) of company.
        Use this when you have to know ROE(Return on Equity) rate of company.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        stock = yf.Ticker(symbol)
        value = str(stock.info['returnOnEquity'] * 100) + "%"

        return f"ROE Rate of {symbol} : {value}"
        
    
    @tool("Get PER rate of company")
    def get_PER(symbol):
        """
        Useful to get PER(Price Earning Ratio) of company.
        Use this when you have to know PER(Price Earning Ratio) rate of company.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        stock = yf.Ticker(symbol)
        value = stock.info['trailingRE']

        return f"PER Rate of {symbol} : {value}"
    
    @tool("Get 5 year's FFO rate of company")
    def get_FFO(symbol):
        """
        Useful to get 5 years FFO of company.
        Use this when you have to know FFO(Funds From Operations) rate of company.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        req = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")

        list_five_years_annual_report = req.json()['annualReports'][0:5]


        FFO = dict()

        for report in list_five_years_annual_report:
            date = report['fiscalDateEnding']
            ffo = float(report['netIncome']) - float(report['otherNonOperatingIncome'])

            FFO[date] = ffo

        return f"5 year's FFO(Including recent FFO) of {symbol} : {FFO}"

    @tool("Get Dividend payout ratio of company")
    def get_diviend_payout_ratio(symbol):
        
        """
        A tool to get Dividend Payout Ratio of company.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        stock = yf.Ticker(symbol)
        value = str(round(stock.info['payoutRatio'] * 100, 2)) + "%"

        return f"Dividend Payout Ratio of {symbol} : {value}"

    @tool("Get Inflation rate")
    def get_inflation_rate():
        """
        A tool to get Inflation rate of US.
        This data is json format.
        """

        inflation = requests.get("https://www.alphavantage.co/query?function=INFLATION&apikey={alpha_vantage_api_key}")

        return inflation.json()
    
    @tool("Get Stock Price")
    def get_stock_price(symbol):
        """
        A tool to get stock price of company.
        This data is very big. You use carefully use this data.
        The input of this tool should be a symbol of company,for example AAPL, NET, TSLA etc...
        """

        stock_price = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol=IBM&apikey={alpha_vantage_api_key}")

        return stock_price.json()["Monthly Adjusted Time Series"]


In [6]:
from crewai import Task

class Tasks:
    
    def symbol_research(self, agent):
        return Task(
            description="""Gather stock symbol of {company}.""",
            expected_output="Your Final Answer MUST be a stock symbol. You MUST NOT find other information about company.",
            agent=agent,
        )
    def indicator_research(self, agent):
        return Task(
            description="Gather information of three company indicator. You should look ROE, PER, FFO rate of {company}. Your job is to order other agents. You should carefully look and give works to appropriate workers.",
            expected_output=f"Your Final Answer must be summary of three company indicator. Ask each three company indicator to each three analyst. You should summarize each of analyst's report.",
            agent=agent,
        )    
    def dividend_payout_ratio_analysis(self, agent):
        return Task(
            description="Get value of dividend payout ratio  of {company}. And judge whether {company} is good to invest",
            expected_output=f"Your Final Answer Must be company's dividend rate and judgement whether this company is good to invest.",
            agent=agent,
        )   
    def news_analysis(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment",
            expected_output=f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance",
            agent=agent,
        )  
    def market_share_analysis(self, agent):
        return Task(
            description="Gather and analyze the market share of {company}. You can use whatever you want",
            expected_output="Your final answer Must be detail summary about market share of company",
            agent=agent,
        ) 
    def stock_price_analyst(self, agent):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions",
            agent=agent,
        )  
    def hedge_fund_manager(self, agent, context):
        return Task(
            description="Based on the response of other agents, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation",
            expected_output=f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports",
            agent=agent,
            context=context,
            output_file="result.md",
        )

In [7]:
from crewai import Agent
from crewai_tools import SerperDevTool, ScrapeWebsiteTool


class Agents:
    def symbol_researcher(self):
        return Agent(
            role="Symbol_Researcher",
            goal="Search and find company's symbol. ",
            backstory="""You're skilled in finding company's stock symbol. You have to find company's stock symbol with company's name. If you cannot find information, just say 'I don't know'. Do not MAKEUP your owns
                        Here are some examples. 
                        Human : What is Apple stock symbol?
                        Your answer : AAPL

                        Human : What is Intel stock symbol?
                        Your answer : INTC

                        Human : What is {company} stock symbol?
                        Your answer :
                    """,
            verbose=True,
            tools = [
                SerperDevTool(),
                ScrapeWebsiteTool()
            ],
            allow_delegation = False

        )  
    def company_indicator_analyst(self):
        return Agent(
            role="Company Indicator Analyst",
            goal="Your task is to act as a 'Coordinator Agent.' You will not solve the problem directly but delegate tasks to specialized agents. These agents will handle different parts of the problem.",
            backstory="""You are talented person to gather the best insights by relying on specialized agents. You should carefully choose agents and give appropriate work to each agents. If you think it is hard to accomplish task, just say I don't know.""",
            verbose=True,
            allow_delegation = True
        )
    
    def ROE_analyst(self):
        return Agent(
            role="ROE(Return on Equity) analyst",
            goal="Analyses ROE(Return on Equity) rate of given company. You should get exact value of Return On Equity rate relying on given tools.",
            backstory="""
                    You are good at summarizing some information. You should get exact value of Return on Equity rate relying on given tools. 
                    Here are some context you can use. Use this context to analysing ROE rate.
                    Context : ROE measures the efficiency and profitability of a company's equity investment. A higher ROE indicates more effective management and profitability, with a benchmark of 15% being desirable. Companies with high ROE are often well-managed and show potential for long-term growth.
                    
                    Finally, you should give recommandation based on your ROE(Return On Equity) analysis. 

                    ==Caution==
                    Do Not MAKEUP.
                    Do Not Calculate your own. Use Tools
                    You MUST only use given context and tools.
                    """,
            verbose = True,
            tools = [
                Tools.get_ROE,
            ],
            allow_delegation = False

            
        )
    
    def PER_analyst(self):
        return Agent(
            role="PER(Price Earning Ratio) analyst",
            goal="Analyses PER(Price Earning Rate) rate of given company. You should get exact value of Price Earning Ratio rate relying on given tools.",
            backstory="""
                    You are good at summarizing some information. You should get exact value of Price Earning Ratio rate relying on given tools. 
                    Here are some context you can use. Use this context to analysing PER rate.
                    Context : PER assesses if a stock is undervalued or overvalued by comparing the stock price to its EPS. A PER below 1 suggests the stock may be undervalued, while a PER above 1 indicates possible overvaluation. Investing in stocks with a PER below 1 is generally seen as a wise choice.
                    
                    Finally, you should give recommandation based on your PER(Price Earning Ratio) analysis. 

                    ==Caution==
                    Do Not MAKEUP.
                    Do Not Calculate your own. Use Tools
                    You MUST only use given context and tools.
                    """,
            verbose = True,
            tools = [
                Tools.get_PER,
            ],
            allow_delegation = False
        )
    
    def FFO_analyst(self):
        return Agent(
            role="FFO(Funds From Operations) analyst",
            goal="Analyses 5 year's FFO(Funds From Operations) rate of given company. You should get exact value of Funds From Operations rate relying on given tools.",
            backstory="""
                    You are good at summarizing some information. You should get exact value of Fund From Operations rate relying on given tools. 
                    Here are some context you can use. Use this context to analysing FFO rate.
                    Context : FFO reflects earnings from core operations by adjusting net income for non-operational gains and expenses. It's a more accurate measure of a company’s operational performance compared to net income. Companies with consistently growing FFO over the past five years are considered good investment opportunities.

                    Finally, you should give recommandation based on your FFO(Funds From Operations) analysis. 

                    ==Caution==
                    Do Not MAKEUP.
                    Do Not Calculate your own. Use Tools
                    You MUST only use given context and tools.
                    """,
            verbose = True,
            tools = [
                Tools.get_FFO,
            ],
            allow_delegation = False,
        )

    
    def dividend_payout_ratio_analyst(self):
        return Agent(
            role="Dividend Payout Ratio Analyst",
            goal="Analyses the dividend payout ratio of company and provides insight of this company. If you cannot find information, just say 'I don't know'. Do not MAKEUP your owns",
            backstory="You are an expert in judging dividend payout ratio. Get dividend payout ratio of company and check out dividend payout ratio is higher than 40%. If dividend payout ratio is higher than 40%, this company is good to invest.",
            verbose=True,
            tools = [
                Tools.get_diviend_payout_ratio,
            ],
            allow_delegation = False
        )
    def news_analyst(self):
        return Agent(
            role="News Analyst",
            goal="Analyses the news of given company. Get the sentiment of each news and analysis if this company is good to invest. You MUST get three recent articles of {company}. If you cannot find information, just say 'I don't know'. Do not MAKEUP your owns",
            backstory="You are an Analyst with news. You analysis each of company news and get sentiment. You should give insight whether it is good to invest this company. Also, return the recent important news of company. You are giving a advice to important customer.",
            verbose=True,
            tools = [
                SerperDevTool(),
                ScrapeWebsiteTool(),
            ],
            allow_delegation = False
        )
    def market_share_analyst(self):
        return Agent(
            role="Market Share Analyst",                            
            goal="Analyses the Market Share of company. You can use any tool to get Market Share of Company. If you cannot find about Market Share, just say 'I don't Know'. Do not MAKEUP your owns.",
            backstory="You are an expert on analysis market share of company. You can use your tools to get information of company Market Share. If you don't know, just I don't know. DO NOT MAKEUP.",
            verbose=True,
            tools = [
            ],
            allow_delegation = False
        )
    def stock_price_analyst(self):
        return Agent(
            role="Stock Price Analyst",
            goal="Analyses the three traditional indicator and provides insight of this company. If you cannot find information, just say 'I don't know'. Do not MAKEUP your owns",
            backstory="You are an expert in Price Analyses. You should judge company stock price with following three statement. First, company's stock price increasement should be higher than market inflation. Second, stock price should not be decrease 15% in recent 1 years. All of this statement is satisfy, this company is good to invest. You should analysis stock price and give response to customer if this company is good to invest.",
            verbose=True,
            tools = [

            ],
            allow_delegation = False
        )
    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            #goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from Traditional Indicator Analyst, Dividend Rate Analyst, News Analyst, Market Share Analyst, Stock Price Analyst. Also, You MUST CONSIDER importance of each data.",
            goal = "Manages a portfolio of stocks and makes strategic investment decisions to maximize returns",
            #backstory="You are the head fund manager. You need to decide whether to buy this company's stock by listening to the opinions of the Traditional Indicator Analyst, Dividend Rate Analyst, News Analyst, Market Share Analyst, and Stock Price Analyst. The importance of each piece of information varies. The importance decreases from left to right, meaning the priority of information is as follows: traditional indicator > dividend rate > news > market share > stock price. Additionally, you are currently dealing with a very important client. ",
            backstory = "You are the head fund manager. You need to decide whether to buy this company's stock by given context",
            verbose=True,
            allow_delegation = False
        )
    

In [ ]:
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

agents = Agents()
task = Tasks()

symbolResearcher = agents.symbol_researcher()
companyIndicatorAnalyst = agents.company_indicator_analyst()
dividendRateAnalyst = agents.dividend_payout_ratio_analyst()
ROE_analyst = agents.ROE_analyst()
PER_analyst = agents.PER_analyst()
FFO_analyst = agents.FFO_analyst()
#news_analyst = agents.news_analyst()
#market_share_analyst = agents.market_share_analyst()
#stock_price_analyst = agents.stock_price_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

symbol_research__Task = task.symbol_research(symbolResearcher)
indicator_research__Task = task.indicator_research(companyIndicatorAnalyst)
dividend_payout_ratio_analysis__Task = task.dividend_payout_ratio_analysis(dividendRateAnalyst)
#news_analysis__Task = task.news_analysis(news_analyst)
#market_share_analysis__Task = task.market_share_analysis(market_share_analyst)
#stock_price_analyst__Task = task.stock_price_analyst(stock_price_analyst)
hedge_fund_manager__Task = task.hedge_fund_manager(hedge_fund_manager, [indicator_research__Task, dividend_payout_ratio_analysis__Task])

list_agents = [symbolResearcher, companyIndicatorAnalyst, ROE_analyst, PER_analyst, FFO_analyst, dividendRateAnalyst, hedge_fund_manager]
list_tasks = [symbol_research__Task, indicator_research__Task, dividend_payout_ratio_analysis__Task, hedge_fund_manager__Task]

crew = Crew(
    agents=list_agents,
    tasks=list_tasks,
    verbose=True,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=True
)

result = crew.kickoff(inputs=dict(company="Intel"))


In [69]:
stock = yf.Ticker("INTC")

stock.income_stmt.loc[['Net Income']]

stock.cash_flow.index


Index(['Free Cash Flow', 'Repurchase Of Capital Stock', 'Repayment Of Debt',
       'Issuance Of Debt', 'Issuance Of Capital Stock', 'Capital Expenditure',
       'Interest Paid Supplemental Data', 'Income Tax Paid Supplemental Data',
       'End Cash Position', 'Beginning Cash Position', 'Changes In Cash',
       'Financing Cash Flow', 'Cash Flow From Continuing Financing Activities',
       'Net Other Financing Charges', 'Proceeds From Stock Option Exercised',
       'Cash Dividends Paid', 'Common Stock Dividend Paid',
       'Net Common Stock Issuance', 'Common Stock Payments',
       'Common Stock Issuance', 'Net Issuance Payments Of Debt',
       'Net Short Term Debt Issuance', 'Short Term Debt Payments',
       'Short Term Debt Issuance', 'Net Long Term Debt Issuance',
       'Long Term Debt Payments', 'Long Term Debt Issuance',
       'Investing Cash Flow', 'Cash Flow From Continuing Investing Activities',
       'Net Other Investing Changes', 'Net Investment Purchase And Sale',